pip install careerjet_api_client

https://github.com/careerjet/careerjet-api-client-python 

In [1]:
import datetime
import time

from careerjet_api_client import CareerjetAPIClient
import pymysql as mdb

In [2]:
time.strftime('%Y-%m-%d')

'2016-01-13'

In [ ]:
# Creating job table

# Multicriteria query

In [3]:
query_keywords = ['data scientist', 
                  'data science',
                  'decision science',
                  'decision scientist',
                  'cognitive',
                  'user research',
                  'user researcher', 
                  'user experience', 
                  'UX',
                  'human computer interaction',
                  'HCI', 
                  'human factor engineer', 
                  'human factor engineering',
                  'usability',
                  'quantitative researcher',
                  'quantitative research',
                  'human factors'
                 ]

query_locations = ['mountain view',
                   'palo alto',
                   'menlo park',
                   'cupertino',
                   'campbell',
                   'san carlos',
                   'san mateo',
                   'sunnyvale', 
                   'santa clara',
                   'san jose',
                   'san francisco'
                  ]

# affid saved in text file
id_file = open('resources/careerjet_affid.txt')
my_id = id_file.readline()
my_id = my_id.strip()

In [7]:
# Each block of 99 results will be stored in a list
all_results = []

cj  =  CareerjetAPIClient("en_US");
for keyword in query_keywords[:1]:
    print keyword + ':'
    for location in query_locations[:1]:
        max_page = 1  # api calculates max num pages based on pagesize and hits
        page_num = 1  # start w/ first page
        total = 0     # running total for this keyword, location 
        while page_num <= max_page:
            time.sleep(1)

            result_json = cj.search({'location'    : location,
                                     'keywords'    : keyword,
                                     'affid'       : my_id,
                                     'user_ip'     : '73.222.121.109',
                                     'url'         : 'http://xfinity.com',
                                     'pagesize'    : 99,
                                     'page'        : page_num,
                                     'contracttype': 'p',
                                     'sort'        : 'salary',
                                     'user_agent'  : 'Chrome/47.0.2526.80 Safari/537.36'
                                      })

            # if bad response, break out of loop
            if 'hits' not in result_json:
                break
                
            # if no jobs whatsoever, move on
            if result_json['hits'] == 0:
                break
            
            # This is the same for each page of query, 
            # and depends on pagesize.
            max_page = result_json['pages']
            
            # Append jobs for this query page
            for job in result_json['jobs']:
                all_results.append((job, keyword, location))

            # Incremement page counter
            page_num += 1
            total += len(result_json['jobs'])
        print '\t', location,
        print total,
        print
    print

print "total:", len(all_results)

data scientist:
	mountain view 30

total: 30


In [13]:
#for job, term, loc in all_results:
#    print job['url']

http://jobviewtrack.com/en-us/job-481e415e4e0d0a00531d53176a050f13042210091d120a585e44480a1b07540021545c5557/547ae473229621697a645ed214ba8f37.html?affid=2dba63bec709301650048b13f60d344d
http://jobviewtrack.com/en-us/job-4b19414a580d4f0a4202450f07110b156744111e0c1f455b585f2f0e0e184607431101111a650f4102091a1058424d590d000a1842184f130d136f5051144d/e6a3a59de6bb473bafe7cf1de460c6bc.html?affid=2dba63bec709301650048b13f60d344d
http://jobviewtrack.com/en-us/job-1f12415e4e0d0a00531d53176a021b1311491a0f49164e4c582f49051b0f07154e0204181a0e0653760c08074b0b4e4e4401011a4e075461090f0f0b1c541d0b1a534e4a494c0d170c07421a540a1b156f5051144d/daacfbb9fd622d02439d80e5e47ef6f3.html?affid=2dba63bec709301650048b13f60d344d
http://jobviewtrack.com/en-us/job-1e1d415e4e0d0a00531d53176a0b0f11042210091d1228415c5b4c440c02480144610c001a06455317010c1d5e424e592c535b5a1e/9a562cbd5579265d13767f2c956def4b.html?affid=2dba63bec709301650048b13f60d344d
http://jobviewtrack.com/en-us/job-4e1e41494c100e4e5417490606020b650f41020949

# Getting this to db

In [6]:
import warnings
warnings.filterwarnings("error")

def removeNonAscii(s):
    return "".join(i for i in s if ord(i)<128)

In [14]:
con = mdb.connect('localhost', 'root', '', 'job_profiler', autocommit=True,
                 charset='utf8', use_unicode=True)
cur = con.cursor()

In [7]:
query_base = "INSERT INTO cj_jobs (salary, salary_min, title, company, locations, site, \
     salary_currency_code, date, salary_max, salary_type, q_term, q_location, url, first_seen, last_seen) \
     VALUES \
    (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"

# Count how many records inserted into SQL table
count = 0

today = time.strftime('%Y-%m-%d')


for job, term, loc in all_results:
    
    try:
        # Safety check inc ase any don't exist
        url = job['url'] if 'url' in job else '' 
        cur.execute("SELECT pid FROM cj_jobs WHERE url = '%s'" % url)
        rows = cur.fetchall()
        if rows:
            cur.execute("UPDATE cj_jobs SET last_seen='%s' WHERE pid='%s'" % (today,rows[0][0]))
            continue

        salary = job['salary'] if 'salary' in job else ''
        salary_min = job['salary_min'] if 'salary_min' in job else ''
        title = removeNonAscii(job['title'].encode('utf8', 'ignore')) if 'title' in job else ''
        company = job['company'] if 'company' in job else ''
        locations = job['locations'] if 'locations' in job else ''
        site = job['site'] if 'site' in job else ''
        salary_currency_code = job['salary_currency_code'] if 'salary_currency_code' in job else ''
        salary_max = job['salary_max'] if 'salary_max' in job else ''
        salary_type = job['salary_type'] if 'salary_type' in job else ''
        date = job['date'] if 'date' in job else ''

        # Convert date time
        date = datetime.datetime.strptime(date, 
                                          '%a, %d %b %Y %H:%M:%S %Z').strftime('%Y-%m-%d')

        # Construct query base
        query_val = (salary, salary_min, title, 
                     company, locations, site,
                     salary_currency_code, date, 
                     salary_max, salary_type, term, loc, url, today, today)

        cur.execute(query_base, query_val)
        count += 1
        
    except RuntimeWarning:
        
        print 'WARNING'
    
print "Finished! Inserted %i rows into table." % count

Finished! Inserted 959 rows into table.
